In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PyHa.statistics import *
from PyHa.IsoAutio import *
from PyHa.visualizations import *
import pandas as pd

## Running Moment to Moment Labeling System on a Directory of WAV Files 

In [31]:
path = "./TEST/"

### Dictionary that defines the isolation technique and parameters used to generate automated labels 

In [32]:
isolation_parameters = {
   "model" : "birdnet",
   "output_path" : "outputs",
   "min_conf" : 0.5,
   "filetype" : "wav", 
   "num_predictions" : 1,
   "write_to_csv" : True
}


# # Example Parameters for Microfaune
# isolation_parameters = {
#      "model" : "microfaune",
#      "technique" : "steinberg",
#      "threshold_type" : "median",
#      "threshold_const" : 2.0,
#      "threshold_min" : 0.0,
#      "window_size" : 2.0,
#      "chunk_size" : 5.0
# }

# Example parameters for TweetyNET
#isolation_parameters = {
#     "model" : "tweetynet",
#     "tweety_output": True,
#     "chunk_size" : 3.0
#}

In [65]:
 %reload_ext autoreload
automated_df = generate_automated_labels(path,isolation_parameters);
automated_df

{'output_path': 'outputs', 'min_conf': 0.5, 'filetype': 'wav', 'num_predictions': 1, 'write_to_csv': True}
PyHa/birdnet_analyzer/eBird_taxonomy_codes_2021E.json
Species list contains 2434 species
Found 11 files to analyze
ScreamingPiha1.wav
Analyzing ./TEST/\ScreamingPiha1.wav
Finished ./TEST/\ScreamingPiha1.wav in 2.08 seconds
ScreamingPiha10.wav
Analyzing ./TEST/\ScreamingPiha10.wav
Finished ./TEST/\ScreamingPiha10.wav in 6.45 seconds
ScreamingPiha11.wav
Analyzing ./TEST/\ScreamingPiha11.wav
Finished ./TEST/\ScreamingPiha11.wav in 4.48 seconds
ScreamingPiha2.wav
Analyzing ./TEST/\ScreamingPiha2.wav
Finished ./TEST/\ScreamingPiha2.wav in 2.25 seconds
ScreamingPiha3.wav
Analyzing ./TEST/\ScreamingPiha3.wav
Finished ./TEST/\ScreamingPiha3.wav in 0.42 seconds
ScreamingPiha4.wav
Analyzing ./TEST/\ScreamingPiha4.wav
Finished ./TEST/\ScreamingPiha4.wav in 0.85 seconds
ScreamingPiha5.wav
Analyzing ./TEST/\ScreamingPiha5.wav
Finished ./TEST/\ScreamingPiha5.wav in 3.49 seconds
ScreamingPiha6.w

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID
0,./TEST/,ScreamingPiha1.wav,71.000816,1,0.0,3.0,48000,Screaming Piha
1,./TEST/,ScreamingPiha1.wav,71.000816,1,3.0,3.0,48000,Screaming Piha
2,./TEST/,ScreamingPiha1.wav,71.000816,1,6.0,3.0,48000,Black-faced Antthrush
3,./TEST/,ScreamingPiha1.wav,71.000816,1,9.0,3.0,48000,Screaming Piha
4,./TEST/,ScreamingPiha1.wav,71.000816,1,9.0,3.0,48000,Golden-crowned Spadebill
...,...,...,...,...,...,...,...,...
7,./TEST/,ScreamingPiha9.wav,81.203537,1,21.0,3.0,48000,Screaming Piha
8,./TEST/,ScreamingPiha9.wav,81.203537,1,24.0,3.0,48000,Screaming Piha
9,./TEST/,ScreamingPiha9.wav,81.203537,1,27.0,3.0,48000,Screaming Piha
10,./TEST/,ScreamingPiha9.wav,81.203537,1,30.0,3.0,48000,Screaming Piha


### Human Manual Labels for Comparison 

In [50]:
manual_df = pd.read_csv("ScreamingPiha_Manual_Labels.csv")
#manual_df = pd.read_csv("BirdCLEF2020_Validation.csv")
manual_df

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID
0,./TEST/,ScreamingPiha1.wav,32.6160,0,1.5448,2.1297,16000,bird
1,./TEST/,ScreamingPiha1.wav,32.6160,0,10.1638,0.8498,16000,bird
2,./TEST/,ScreamingPiha1.wav,32.6160,0,0.5549,0.9999,16000,bird
3,./TEST/,ScreamingPiha1.wav,32.6160,0,8.7739,0.8399,16000,bird
4,./TEST/,ScreamingPiha1.wav,32.6160,0,12.6335,1.9997,16000,bird
...,...,...,...,...,...,...,...,...
249,./TEST/,ScreamingPiha2.wav,33.9331,0,26.9274,1.7602,44100,bird
250,./TEST/,ScreamingPiha2.wav,33.9331,0,30.8178,0.7200,44100,bird
251,./TEST/,ScreamingPiha2.wav,33.9331,0,29.8677,0.9401,44100,bird
252,./TEST/,ScreamingPiha2.wav,33.9331,0,31.5378,1.9502,44100,bird


### Pandas Output of Automated Labeling 

In [ ]:
automated_df

### Function that gathers statistics about the duration of annotations

In [ ]:
annotation_duration_statistics(automated_df)

In [ ]:
annotation_duration_statistics(manual_df)

### Histogram Visualization of Annotation Lengths

In [ ]:
annotation_duration_histogram(automated_df, n_bins = 100, title = "Automated Annotation Lengths")

In [ ]:
annotation_duration_histogram(manual_df, n_bins = 100, title = "Human Annotation Lengths")
# In the Screaming Piha clips we can see two distinct clusters based on the "Whoa" vocalizations (.25-1.25s)
# and the "Pee-Haw" vocalizations (1.25-2.5s)

### Helper function to convert to kaleidoscope-compatible format 

In [ ]:
kaleidoscope_conversion(manual_df)

## Different Combinations of Spectrogram and Local Score Visualization Graphs
Note: Not all models support local score arrays.

### Baseline Graph without any annotations

In [ ]:
clip_path = "./TEST/ScreamingPiha2.wav"
spectrogram_visualization(clip_path)

### Baseline graph with log scale

In [ ]:
spectrogram_visualization(clip_path, isolation_parameters = isolation_parameters, log_scale = True)

### Baseline graph with normalized local score values between [0,1] 

In [ ]:
spectrogram_visualization(clip_path, isolation_parameters = isolation_parameters, normalize_local_scores = True)

### Graph with Automated Labeling 

In [ ]:
spectrogram_visualization(clip_path,automated_df = True, isolation_parameters = isolation_parameters)

### Spectrogram with Human Labelling

In [ ]:
spectrogram_visualization(clip_path, premade_annotations_df = manual_df[manual_df["IN FILE"] == "ScreamingPiha2.wav"],premade_annotations_label = "Piha Human Labels")

### Graph with Both Automated and Human Labels 

In [ ]:
spectrogram_visualization(clip_path,automated_df = True,isolation_parameters=isolation_parameters,premade_annotations_df = manual_df[manual_df["IN FILE"] == "ScreamingPiha2.wav"])

### Another Visualization of True Positives, False Positives, False Negatives, and True Negatives 

In [ ]:
automated_piha_df = automated_df[automated_df["IN FILE"] == "ScreamingPiha2.wav"]
manual_piha_df = manual_df[manual_df["IN FILE"] == "ScreamingPiha2.wav"]
piha_stats = binary_visualization(automated_piha_df,manual_piha_df)

### Function that generates statistics to gauge efficacy of automated labeling compared to human labels 

In [ ]:
# step necessary to make the birdnet_labels binary.
automated_df['MANUAL ID']='bird'

In [ ]:
clip_statistics_df = clip_statistics(automated_df,manual_df, stats_type = "general")
clip_statistics_df

### Function that takes the statistical ouput of all of the clips and gets the equivalent global scores 

In [ ]:
global_statistics(clip_statistics_df)

### Function that takes in the manual and automated labels for a clip and outputs human label-by-label IoU Scores. Used to derive statistics that measure how well a system is isolating desired segments of audio clips

In [ ]:
Intersection_over_Union_Matrix = clip_IoU(automated_piha_df,manual_piha_df)
print(Intersection_over_Union_Matrix)

### Function that turns the IoU Matrix of a clip into true positive and false positives values, as well as computing the precision, recall, and F1 statistics

In [ ]:
matrix_IoU_Scores(Intersection_over_Union_Matrix,manual_piha_df,0.5)

### Wrapper function that takes matrix_IoU_Scores across multiple clips. Allows user to modify the threshold that determines whether or not a label is a true positive.

In [ ]:
clip_stats_df = clip_statistics(automated_df,manual_df,stats_type = "IoU", threshold = 0.5)
clip_stats_df

### Function that takes the output of dataset_IoU Statistics and ouputs a global count of true positives and false positives, as well as computing common metrics across the dataset

In [ ]:
global_stats_df = global_statistics(clip_stats_df)
global_stats_df